<a href="https://www.kaggle.com/code/xshaimaa/medical-appointment-dataset-analysis" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

## Dataset Description 
A person makes a doctor appointment, receives all the instructions and no-show. Who to blame?
This dataset collects information from 100k medical appointments in Brazil and is focused on the question of whether or not patients show up for their appointment. A number of characteristics about the patient are included in each row.
 
 
## Columns Description
1. `PatientId`: Identification of a patient.
2. `AppointmentID`: Identification of each appointment.
3. `Gender`: Male or Female.
4. `AppointmentDay`: The day of the actuall appointment, when they have to visit the doctor.
5. `ScheduledDay`: The day someone called or registered the appointment, this is before appointment of course.
6. `Age`: How old is the patient.
7. `Neighbourhood`: Where the appointment takes place.
8. `Scholarship`: True of False, indicates whether or not the patient is enrolled in Brasilian welfare program Bolsa Família.
9. `Hipertension`: True or False.
10. `Diabetes`: True or False.
11. `Alcoholism`: True or False.
12. `Handcap`: True or False.
13. `SMS_received`: 1 or more messages sent to the patient.
14. `No-show`: True (if the patient did not show up), or False (if the patient did show up).


## EDA Questions
- What factors are important for us to know in order to predict if a patient will show up for their scheduled appointment?
___

## Environment set-up

In [1]:
# importing lib.
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt
import seaborn as sns

# getting the csv file directory
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

## Data Wrangling

in this section, we'd load our data from a CSV file to a pandas dataframe, and then take a quick dive into exploring our dataset in details.

In [2]:
# loading dataset from csv file and showing its first 5 rows
df = pd.read_csv('/kaggle/input/noshowappointments/KaggleV2-May-2016.csv')
df.head()

We'll move next into exploring our dataset by going through its data types, NaNs or duplicated rows, and any columns that may need to be dropped or parsed.

In [3]:
# viewing main info about df
df.info()

- we can notice there are no NaNs at all in our data
- `PatientId` and `AppointmentId` columns wouldn't be helpful during analysis.
- `ScheduledDay` and `AppointmentDay` needs to be casted to date data type.
- we may append a new column for days until appointment.
- `Gender` needs to be converted into a categoy type
- `Scholarship` `Hipertension` `Diabetes` `Alcoholism` `Handcap` better be boolean data type.
- `No-show` needs to be parsed and casted to boolean too.

In [4]:
# checking for duplicates
df.duplicated().sum()

- our dataset has no duplicated rows either.

In [5]:
# exploring the unique values of each column
df.nunique()

- `Handcap` and `Age` columns has inconsistant unique values.
- `SMS_received` would be casted to boolean data type.

In [6]:
# exploring handcap values
df['Handcap'].value_counts()

- we'd be only intrested in rows with `0` or `1` values.

In [7]:
# exploring age column distribution
df['Age'].describe()

- `Age` column would need to be handled.

## Exploration Summery
1. our dataset consists of 110527 rows with 14 columns, and has no NaNs nor duplicated values.
2. `PatientId` and `AppointmentId` columns wouldn't be helpful during analysis.
3. `ScheduledDay` and `AppointmentDay` needs to be casted to date data type.
4. we may append a new column for days until appointment.
5. `Gender` needs to be casted into a categoy type
6. `Scholarship`, `Hipertension`, `Diabetes`, `Alcoholism` and `SMS_recieved` better be boolean data type.
7. `No-show` column needs to be parsed and asted to boolean type.
8. `Handcap` colume needs to be cleaned to have only `0` and `1` values.
9. `Age` columns has inconsistant unique values that needs to be handled.
___

## Data Cleaning
in this section, we'd perform some operations on our dataset based on the previous findings to make our analysis more accurate and clear.

___
**Dropping `PatientId` and `AppointmentId` columns**

In [8]:
# dropping columns and validating changes
df.drop(['PatientId', 'AppointmentID'], axis = 1, inplace = True)
df.columns

___
**Handling `date` data type**

In [9]:
df.AppointmentDay.unique

it looks like all hours are set to 00:00:00, so we would want to extract onl the year, month and day data

In [10]:
# extracting only day, month and year values
df['ScheduledDay'] = df['ScheduledDay'].str[:10]
df['AppointmentDay'] = df['AppointmentDay'].str[:10]

# changing data type
df['ScheduledDay'] = pd.to_datetime(df['ScheduledDay'])
df['AppointmentDay'] = pd.to_datetime(df['AppointmentDay'])

# confirming changes
print(df[['AppointmentDay', 'ScheduledDay']].dtypes)
df.head()

Now, we'd move into appending a new column that holds number of days to the appointment.

In [11]:
# making new due days column
df['due-days'] = df['AppointmentDay'] - df['ScheduledDay']

# converting data type 
df['due-days'] = df['due-days'].dt.days

# drop sch and appoint col
df.drop(['AppointmentDay', 'ScheduledDay'], axis = 1, inplace = True)

We'll move into exploring this new column.

In [12]:
# viewing summery statistics
df['due-days'].describe()

We seem to have some negative values here, we'll drop them.

In [13]:
# viewing negative days values
df[df['due-days'] < 0 ]

In [14]:
# dropping these values and confirming changes
df.drop(df[df['due-days'] < 0].index, inplace = True)
df['due-days'].describe()

___
**Converting `Gender` and `No-show` to categorical variables**

In [15]:
# converting column and confirming changes
df['Gender'] = df['Gender'].astype('category')

df['Gender'].dtypes

___
**Converting `Scholarship`, `Hipertension`, `Diabetes`, `Alcoholism`, `Handcap` and `SMS_recieved` to boolean data type**

In [16]:
# converting columns to bool and confirming changes
cols = ['Scholarship', 'Hipertension', 'Diabetes', 'Alcoholism', 'SMS_received']
df[cols] = df[cols].astype('bool')
df[cols].dtypes

___
**Parsing and casting `No-show` column**

In [17]:
# mapping alues to be more familiar
df.loc[df['No-show'] == 'Yes', 'No-show'] = 0
df.loc[df['No-show'] == 'No', 'No-show'] = 1

# casting dt type and confirming changes
df['No-show'] = df['No-show'].astype(bool)
df['No-show'].dtypes

___
**Cleaning `Handcap` column**

In [18]:
# viewing rows with values of handcap > 1
df[df['Handcap'] > 1]

We have 199 rows with inconsistant values, we'd replace them with 1 to treat them as beeing handcaped

In [19]:
# filling the bigger values with 1
df.loc[df['Handcap'].isin([2, 3, 4]), 'Handcap'] = 1

# casting type and confirming changes
df['Handcap'] = df['Handcap'].astype('bool')
df['Handcap'].unique()

___
**Cleaning `Age` column**

In [20]:
#exploring values below 0
df[df['Age'] < 0]

- we have one value with negative age, so we will drop it

In [21]:
# dropping row with negative age and confirming changes
df.drop(df[df['Age'] < 0].index, inplace = True)
df[df['Age'] < 0]

___

In [22]:
df.head()

In [24]:
df.info()

We endded up with a datafram of 110521 rows and 11 columns, and everything looks tidy and clean. We'd proceed in visualizing it to extract meaningful insights from it.
___